# Week 5 Quiz

In [3]:
import numpy as np

In [30]:
def prior_energy(S, beta):
    return beta * ((np.diff(S, axis=0)!=0).sum() + (np.diff(S, axis=1)!=0).sum())   
    
def likelihood_energy(S, I, mu):
    return ((mu[S] - I)**2).sum()

We consider the image

In [4]:
I = np.array([
    [1,2,6,4,10,8],
    [4,1,3,5,9,6],
    [5,2,3,5,4,7]
])

which is to be segmented into three segments: 1, 2 and 3. The segments are estimated
to have mean intensities µ1 = 2, µ2 = 5 and µ3 = 10. We use an MRF framework as
in the lecture note, and we choose to set β = 10.

In [5]:
mu = np.array([2, 5, 10])
beta = 10

In [18]:
#Likelihood
U = np.stack([mu[0] - I, mu[1] - I, mu[2] - I], axis=2)**2
U.shape

(3, 6, 3)

In [22]:
S0 = np.argmin(U, axis=2)
S0

array([[0, 0, 1, 1, 2, 2],
       [1, 0, 0, 1, 2, 1],
       [1, 0, 0, 1, 1, 1]])

## 1)
What is the prior energy of the maximum likelihood solution?

In [23]:
# Prior
prior = beta * ((S0[:-1] != S0[1:]).sum() + (S0[:,:-1] != S0[:,1:]).sum())
prior

120

# 2)
What is the likelihood energy of the configuration where the leftmost two columns
are labeled as belonging to the segment 1, middle two columns are labeled as belonging to the segment 2, and rightmost two colums are labeled as belonging to
the segment 3?

In [24]:
new_S0 = np.array([[0, 0, 1, 1, 2, 2],
                    [0, 0, 1, 1, 2, 2],
                    [0, 0, 1, 1, 2, 2]])

In [25]:
#Likelihood
likelihood = ((mu[new_S0] - I)**2).sum()
likelihood

91

# 3)
What is the posterior energy of the maximum a posteriory solution? (Write the
smallest energy, which you could find.)

Testing a posteriory with the new mask

In [31]:
# prior
new_prior = beta * ((new_S0[:-1] != new_S0[1:]).sum() + (new_S0[:,:-1] != new_S0[:,1:]).sum())
new_prior

60

In [27]:
likelihood + new_prior

151

Finding optimal (maximum a posteriory) solution using graph cuts

In [28]:
import maxflow.fastmin

In [33]:
S_GC = S0.copy()
maxflow.fastmin.aexpansion_grid(U, beta - beta*np.eye(3, 3, dtype=U.dtype), labels = S_GC)
posterior_GC  = prior_energy(S_GC, beta) + likelihood_energy(S_GC, I, mu)
print(f'posterior_GC: {posterior_GC}')

posterior_GC: 111
